# Lora 实战

## Step1 导入相关包

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer
import pandas as pd

## Step2 加载数据集

In [ ]:
# 将JSON文件转换为CSV文件
df = pd.read_json('dataset/paimeng.json')
ds = Dataset.from_pandas(df)

In [ ]:
ds[:3]

## Step3 数据集预处理

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("model/chatglm3-6b", 
                                          trust_remote_code=True)
tokenizer

In [ ]:
print(tokenizer.encode(ds[0]['instruction']))

In [ ]:
demo_token = tokenizer.build_single_message('system', "", "现在你要扮演原神开放世界中探险的向导--派蒙")
demo_token, tokenizer.decode(demo_token)

In [ ]:
tokenizer.get_command("[gMASK]"), tokenizer._convert_id_to_token(+tokenizer.eos_token_id)

### 调用api处理数据

In [ ]:
instruction = "\n".join([ds[0]["instruction"], ds[0]["input"]]).strip()     # query
instruction = tokenizer.build_chat_input(instruction, history=[], role="user")
response = tokenizer("\n" + ds[0]["output"], add_special_tokens=False)
input_ids = instruction["input_ids"][0].numpy().tolist() + response["input_ids"] + [tokenizer.eos_token_id]
tokenizer.decode(input_ids)

### 手动拆解数据处理

In [ ]:
prompt = [tokenizer.get_command("<|system|>")] + tokenizer.encode("现在你要扮演原神开放世界中探险的向导--派蒙\n ", add_special_tokens=False)
instruction_ = [tokenizer.get_command("<|user|>")] + tokenizer.encode("\n " + "\n".join([ds[0]["instruction"], ds[0]["input"]]).strip(), add_special_tokens=False,max_length=512) + [tokenizer.get_command("<|assistant|>")]
instruction = tokenizer.encode(prompt + instruction_)
response = tokenizer.encode("\n" + ds[0]["output"], add_special_tokens=False)
input_ids = instruction + response + [tokenizer.eos_token_id]
tokenizer.decode(input_ids)

In [ ]:
def process_func(example):
    MAX_LENGTH = 512
    input_ids, labels = [], []
    prompt = [tokenizer.get_command("<|system|>")] + tokenizer.encode("现在你要扮演原神开放世界中探险的向导--派蒙\n ", add_special_tokens=False)
    instruction_ = [tokenizer.get_command("<|user|>")] + tokenizer.encode("\n " + "\n".join([example["instruction"], example["input"]]).strip(), add_special_tokens=False,max_length=512) + [tokenizer.get_command("<|assistant|>")]
    instruction = tokenizer.encode(prompt + instruction_)
    response = tokenizer.encode("\n" + example["output"], add_special_tokens=False)
    input_ids = instruction + response + [tokenizer.eos_token_id]
    labels = [tokenizer.pad_token_id] * len(instruction) + response + [tokenizer.eos_token_id]
    pad_len = MAX_LENGTH - len(input_ids)
    # print()
    input_ids += [tokenizer.pad_token_id] * pad_len
    labels += [tokenizer.pad_token_id] * pad_len
    labels = [(l if l != tokenizer.pad_token_id else -100) for l in labels]

    return {
        "input_ids": input_ids,
        "labels": labels
    }

In [ ]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

In [ ]:
tokenizer.decode(tokenized_ds[1]["input_ids"])

In [ ]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

## Step4 创建模型

In [ ]:
model = AutoModelForCausalLM.from_pretrained("model/chatglm3-6b", 
                                             trust_remote_code=True, 
                                             low_cpu_mem_usage=True)

## Lora

### PEFT Step1 配置文件

- target_modules也可以传入正则项,比如以h.1结尾的query_key_value：".*\.1.*query_key_value"  
- modules_to_save指定的是除了拆成lora的模块，其他的模块可以完整的指定训练。

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model

# model = AutoModelForCausalLM.from_pretrained("/CV/xhr/xhr_project/LLM_learn/transformers-code-master/model/chatglm3-6b", low_cpu_mem_usage=True)
config = LoraConfig(task_type=TaskType.CAUSAL_LM, 
                    target_modules={"query_key_value"}, 
                    r=8, 
                    lora_alpha=32)
config

### PEFT Step2 创建模型

In [ ]:
model = get_peft_model(model, config)

In [ ]:
model.print_trainable_parameters()

## Step5 配置训练参数

In [ ]:
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100,
    pad_to_multiple_of=None,
    padding=False
)

In [ ]:
args = TrainingArguments(
    output_dir="output/ChatGLM",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=20,
    num_train_epochs=1
)

## Step6 创建训练器

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds,
    data_collator=data_collator,
)

## Step7 模型训练

In [ ]:
trainer.train()

## Step8 模型推理

In [ ]:
model = model.cuda()
ipt = tokenizer("<|system|>\n现在你要扮演原神开放世界中探险的向导--派蒙\n<|user|>\n {}\n{}".format("你是谁？", "").strip() + "<|assistant|>\n", return_tensors="pt").to(model.device)
tokenizer.decode(model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True)